## Second Notebook Dealing with Intraday Demand

# Was having issues with the merge on the forecastdemand data - looks like there were duplciates in the original DF that was causing issues. 

In [62]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression  
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
forecastdemand_vic = pd.read_csv("../data/Australia/a/forecastdemand_vic.csv")
forecastdemand_sa = pd.read_csv("../data/Australia/b/forecastdemand_sa.csv")
forecastdemand_qld = pd.read_csv("../data/Australia/c/forecastdemand_qld.csv")
temperature_qld = pd.read_csv("../data/Australia/d/temprature_qld.csv")  # There's a typo in 'temperature' in the file path
temperature_sa = pd.read_csv("../data/Australia/d/temprature_sa.csv")  # Same typo as above
temperature_vic = pd.read_csv("../data/Australia/d/temprature_vic.csv")  # Same typo as above
totaldemand_qld = pd.read_csv("../data/Australia/d/totaldemand_qld.csv")
totaldemand_sa = pd.read_csv("../data/Australia/d/totaldemand_sa.csv")
totaldemand_vic = pd.read_csv("../data/Australia/d/totaldemand_vic.csv")

In [ ]:
dataframes = [forecastdemand_qld, forecastdemand_vic, totaldemand_sa, totaldemand_qld, temperature_vic, temperature_sa, temperature_qld, totaldemand_vic]

for df in dataframes:
    df['DATETIME'] = pd.to_datetime(df['DATETIME'])

for df in dataframes:
    print(df.head())

In [5]:
forecastdemand_qld['DATETIME'] = pd.to_datetime(forecastdemand_qld['DATETIME'])
temperature_qld['DATETIME'] = pd.to_datetime(temperature_qld['DATETIME'])
totaldemand_qld['DATETIME'] = pd.to_datetime(totaldemand_qld['DATETIME'])

merged_df_1 = pd.merge(forecastdemand_qld, temperature_qld, on='DATETIME', how='inner')
df_QLD = pd.merge(merged_df_1, totaldemand_qld, on='DATETIME', how='inner')


df_QLD['forecast_error'] = df_QLD['FORECASTDEMAND'] - df_QLD['TOTALDEMAND']
df_QLD['forecast_error_ratio'] = df_QLD['forecast_error'] / df_QLD['FORECASTDEMAND']

print(df_QLD.head(10))

   PREDISPATCHSEQNO REGIONID_x  PERIODID  FORECASTDEMAND          LASTCHANGED  \
0        2016123018       QLD1        71         6425.36  2016-12-30 12:31:25   
1        2016123019       QLD1        70         6425.42  2016-12-30 13:01:33   
2        2016123020       QLD1        69         6425.42  2016-12-30 13:31:24   
3        2016123021       QLD1        68         6427.40  2016-12-30 14:01:21   
4        2016123022       QLD1        67         6426.99  2016-12-30 14:31:22   
5        2016123023       QLD1        66         6426.63  2016-12-30 15:01:26   
6        2016123024       QLD1        65         6426.63  2016-12-30 15:31:28   
7        2016123025       QLD1        64         6425.73  2016-12-30 16:01:13   
8        2016123026       QLD1        63         6426.89  2016-12-30 16:31:13   
9        2016123027       QLD1        62         6426.80  2016-12-30 17:01:13   

    DATETIME                      LOCATION  TEMPERATURE  TOTALDEMAND  \
0 2017-01-01  Brisbane Archerfield A

In [7]:
from functools import reduce
dataframes_qld = [forecastdemand_qld, totaldemand_qld, temperature_qld]
df_qld_merged = reduce(lambda left, right: pd.merge(left, right, on='DATETIME', how='inner'), dataframes_qld)
print(df_qld_merged.head())



   PREDISPATCHSEQNO REGIONID_x  PERIODID  FORECASTDEMAND          LASTCHANGED  \
0        2016123018       QLD1        71         6425.36  2016-12-30 12:31:25   
1        2016123019       QLD1        70         6425.42  2016-12-30 13:01:33   
2        2016123020       QLD1        69         6425.42  2016-12-30 13:31:24   
3        2016123021       QLD1        68         6427.40  2016-12-30 14:01:21   
4        2016123022       QLD1        67         6426.99  2016-12-30 14:31:22   

    DATETIME  TOTALDEMAND REGIONID_y                      LOCATION  \
0 2017-01-01      6728.99       QLD1  Brisbane Archerfield Airport   
1 2017-01-01      6728.99       QLD1  Brisbane Archerfield Airport   
2 2017-01-01      6728.99       QLD1  Brisbane Archerfield Airport   
3 2017-01-01      6728.99       QLD1  Brisbane Archerfield Airport   
4 2017-01-01      6728.99       QLD1  Brisbane Archerfield Airport   

   TEMPERATURE  
0         26.5  
1         26.5  
2         26.5  
3         26.5  
4      

In [8]:
dataframes_qld = [forecastdemand_qld, totaldemand_qld, temperature_qld]

for df in dataframes_qld:
    df['YEAR'] = df['DATETIME'].dt.year
    df['MONTH'] = df['DATETIME'].dt.month
    df['DAY'] = df['DATETIME'].dt.day
    df['HOUR'] = df['DATETIME'].dt.hour
    df['MINUTE'] = df['DATETIME'].dt.minute

merge_keys = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']
df_qld_merged = reduce(lambda left, right: pd.merge(left, right, on=merge_keys, how='inner'), dataframes_qld)

print(df_qld_merged.head())



   PREDISPATCHSEQNO REGIONID_x  PERIODID  FORECASTDEMAND          LASTCHANGED  \
0        2016123018       QLD1        71         6425.36  2016-12-30 12:31:25   
1        2016123019       QLD1        70         6425.42  2016-12-30 13:01:33   
2        2016123020       QLD1        69         6425.42  2016-12-30 13:31:24   
3        2016123021       QLD1        68         6427.40  2016-12-30 14:01:21   
4        2016123022       QLD1        67         6426.99  2016-12-30 14:31:22   

  DATETIME_x  YEAR  MONTH  DAY  HOUR  MINUTE DATETIME_y  TOTALDEMAND  \
0 2017-01-01  2017      1    1     0       0 2017-01-01      6728.99   
1 2017-01-01  2017      1    1     0       0 2017-01-01      6728.99   
2 2017-01-01  2017      1    1     0       0 2017-01-01      6728.99   
3 2017-01-01  2017      1    1     0       0 2017-01-01      6728.99   
4 2017-01-01  2017      1    1     0       0 2017-01-01      6728.99   

  REGIONID_y                      LOCATION   DATETIME  TEMPERATURE  
0       QLD

In [9]:
print("Forecast Demand QLD 'DATETIME' Sample and Dtype:")
print(forecastdemand_qld['DATETIME'].head())
print(forecastdemand_qld['DATETIME'].dtype)

print("\nTemperature QLD 'DATETIME' Sample and Dtype:")
print(temperature_qld['DATETIME'].head())
print(temperature_qld['DATETIME'].dtype)

print("\nTotal Demand QLD 'DATETIME' Sample and Dtype:")
print(totaldemand_qld['DATETIME'].head())
print(totaldemand_qld['DATETIME'].dtype)


Forecast Demand QLD 'DATETIME' Sample and Dtype:
0   2017-01-01 00:00:00
1   2017-01-01 00:30:00
2   2017-01-01 01:00:00
3   2017-01-01 01:30:00
4   2017-01-01 02:00:00
Name: DATETIME, dtype: datetime64[ns]
datetime64[ns]

Temperature QLD 'DATETIME' Sample and Dtype:
0   2010-01-01 00:00:00
1   2010-01-01 00:30:00
2   2010-01-01 01:00:00
3   2010-01-01 01:30:00
4   2010-01-01 02:00:00
Name: DATETIME, dtype: datetime64[ns]
datetime64[ns]

Total Demand QLD 'DATETIME' Sample and Dtype:
0   2010-01-01 00:00:00
1   2010-01-01 00:30:00
2   2010-01-01 01:00:00
3   2010-01-01 01:30:00
4   2010-01-01 02:00:00
Name: DATETIME, dtype: datetime64[ns]
datetime64[ns]


In [14]:
unique_forecastdemand = forecastdemand_qld['DATETIME'].nunique()
unique_temperature = temperature_qld['DATETIME'].nunique()
unique_totaldemand = totaldemand_qld['DATETIME'].nunique()

print(f"Unique DATETIME in forecastdemand_qld: {unique_forecastdemand}")
print(f"Unique DATETIME in temperature_qld: {unique_temperature}")
print(f"Unique DATETIME in totaldemand_qld: {unique_totaldemand}")


print(forecastdemand_qld.shape)
print(totaldemand_qld.shape)
print(temperature_qld.shape)


Unique DATETIME in forecastdemand_qld: 73833
Unique DATETIME in temperature_qld: 208085
Unique DATETIME in totaldemand_qld: 196513
(4095592, 11)
(196513, 8)
(208085, 8)


In [15]:
datetime_counts_forecastdemand = forecastdemand_qld['DATETIME'].value_counts()
print(datetime_counts_forecastdemand.head(10))
print("\nDistribution of DATETIME occurrence counts:")
print(datetime_counts_forecastdemand.describe())


2019-08-02 04:00:00    79
2020-03-04 04:00:00    79
2018-04-10 04:00:00    79
2019-01-02 04:00:00    79
2019-11-17 04:00:00    79
2020-11-22 04:00:00    79
2017-10-15 04:00:00    79
2018-08-30 04:00:00    79
2019-09-05 04:00:00    79
2020-07-20 04:00:00    79
Name: DATETIME, dtype: int64

Distribution of DATETIME occurrence counts:
count    73833.000000
mean        55.471022
std         13.871803
min         23.000000
25%         43.000000
50%         55.000000
75%         67.000000
max         79.000000
Name: DATETIME, dtype: float64


In [16]:
specific_datetime_df = forecastdemand_qld[forecastdemand_qld['DATETIME'] == '2019-08-02 04:00:00']
print(specific_datetime_df)


         PREDISPATCHSEQNO REGIONID  PERIODID  FORECASTDEMAND  \
2509378        2019073118     QLD1        79         5199.24   
2509435        2019073119     QLD1        78         5196.16   
2509492        2019073120     QLD1        77         5192.53   
2509549        2019073121     QLD1        76         5192.46   
2509606        2019073122     QLD1        75         5192.41   
...                   ...      ...       ...             ...   
2513466        2019080144     QLD1         5         5129.04   
2513518        2019080145     QLD1         4         5121.99   
2513569        2019080146     QLD1         3         5127.00   
2513619        2019080147     QLD1         2         5124.00   
2513668        2019080148     QLD1         1         5107.55   

                 LASTCHANGED            DATETIME  YEAR  MONTH  DAY  HOUR  \
2509378  2019-07-31 12:31:29 2019-08-02 04:00:00  2019      8    2     4   
2509435  2019-07-31 13:01:34 2019-08-02 04:00:00  2019      8    2     4   
250

In [17]:
forecastdemand_qld.drop_duplicates(subset='DATETIME', inplace=True)
print(forecastdemand_qld.shape)


(73833, 11)


In [29]:
dataframes_qld = [forecastdemand_qld, totaldemand_qld, temperature_qld]

for df in dataframes_qld:
    df['YEAR'] = df['DATETIME'].dt.year
    df['MONTH'] = df['DATETIME'].dt.month
    df['DAY'] = df['DATETIME'].dt.day
    df['HOUR'] = df['DATETIME'].dt.hour
    df['MINUTE'] = df['DATETIME'].dt.minute

merge_keys = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']
df_qld_merged = reduce(lambda left, right: pd.merge(left, right, on=merge_keys, how='inner'), dataframes_qld)

print(df_qld_merged.head())

   PREDISPATCHSEQNO REGIONID_x  PERIODID  FORECASTDEMAND          LASTCHANGED  \
0        2016123018       QLD1        71         6425.36  2016-12-30 12:31:25   
1        2016123018       QLD1        72         6193.34  2016-12-30 12:31:25   
2        2016123018       QLD1        73         6016.45  2016-12-30 12:31:25   
3        2016123018       QLD1        74         5859.23  2016-12-30 12:31:25   
4        2016123018       QLD1        75         5767.15  2016-12-30 12:31:25   

           DATETIME_x  YEAR  MONTH  DAY  HOUR  MINUTE          DATETIME_y  \
0 2017-01-01 00:00:00  2017      1    1     0       0 2017-01-01 00:00:00   
1 2017-01-01 00:30:00  2017      1    1     0      30 2017-01-01 00:30:00   
2 2017-01-01 01:00:00  2017      1    1     1       0 2017-01-01 01:00:00   
3 2017-01-01 01:30:00  2017      1    1     1      30 2017-01-01 01:30:00   
4 2017-01-01 02:00:00  2017      1    1     2       0 2017-01-01 02:00:00   

   TOTALDEMAND REGIONID_y                      LOC

In [30]:
df_qld_merged.columns

Index(['PREDISPATCHSEQNO', 'REGIONID_x', 'PERIODID', 'FORECASTDEMAND',
       'LASTCHANGED', 'DATETIME_x', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE',
       'DATETIME_y', 'TOTALDEMAND', 'REGIONID_y', 'LOCATION', 'DATETIME',
       'TEMPERATURE'],
      dtype='object')

In [31]:
df_qld_merged.drop(columns=['REGIONID_x', 'DATETIME_x'], inplace=True)
print(df_qld_merged.columns)


Index(['PREDISPATCHSEQNO', 'PERIODID', 'FORECASTDEMAND', 'LASTCHANGED', 'YEAR',
       'MONTH', 'DAY', 'HOUR', 'MINUTE', 'DATETIME_y', 'TOTALDEMAND',
       'REGIONID_y', 'LOCATION', 'DATETIME', 'TEMPERATURE'],
      dtype='object')


In [32]:
df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,TOTALDEMAND,REGIONID_y,LOCATION,DATETIME,TEMPERATURE
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,6728.99,QLD1,Brisbane Archerfield Airport,2017-01-01 00:00:00,26.5
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,6462.14,QLD1,Brisbane Archerfield Airport,2017-01-01 00:30:00,26.3
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,6352.82,QLD1,Brisbane Archerfield Airport,2017-01-01 01:00:00,26.0
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,6261.02,QLD1,Brisbane Archerfield Airport,2017-01-01 01:30:00,25.2
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,6169.66,QLD1,Brisbane Archerfield Airport,2017-01-01 02:00:00,25.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72451,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,6443.62,QLD1,Brisbane Archerfield Airport,2021-03-17 22:00:00,19.6
72452,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,6264.63,QLD1,Brisbane Archerfield Airport,2021-03-17 22:30:00,19.5
72453,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,6144.16,QLD1,Brisbane Archerfield Airport,2021-03-17 23:00:00,19.4
72454,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,5897.64,QLD1,Brisbane Archerfield Airport,2021-03-17 23:30:00,19.6


In [33]:
df_qld_merged['is_weekend'] = df_qld_merged['DATETIME'].dt.dayofweek >= 5

df_qld_merged['season'] = df_qld_merged['DATETIME'].dt.month % 12 // 3
df_qld_merged['season'] = df_qld_merged['season'].map({0: 'Summer', 1: 'Autumn', 2: 'Winter', 3: 'Spring'}, na_action='ignore')


australian_seasons = {
    12: 'Summer', 1: 'Summer', 2: 'Summer',
    3: 'Autumn', 4: 'Autumn', 5: 'Autumn',
    6: 'Winter', 7: 'Winter', 8: 'Winter',
    9: 'Spring', 10: 'Spring', 11: 'Spring'
    }
df_qld_merged['season'] = df_qld_merged['MONTH'].map(australian_seasons)

In [27]:
#df_qld_merged = df_qld_merged.sort_values('DATETIME')
#df_qld_merged['demand_lag1'] = df_qld_merged['TOTALDEMAND'].shift(24)
#df_qld_merged['rolling_mean_24h'] = df_qld_merged['TOTALDEMAND'].rolling(window=24).mean()

In [34]:
df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,TOTALDEMAND,REGIONID_y,LOCATION,DATETIME,TEMPERATURE,is_weekend,season
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,6728.99,QLD1,Brisbane Archerfield Airport,2017-01-01 00:00:00,26.5,True,Summer
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,6462.14,QLD1,Brisbane Archerfield Airport,2017-01-01 00:30:00,26.3,True,Summer
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,6352.82,QLD1,Brisbane Archerfield Airport,2017-01-01 01:00:00,26.0,True,Summer
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,6261.02,QLD1,Brisbane Archerfield Airport,2017-01-01 01:30:00,25.2,True,Summer
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,6169.66,QLD1,Brisbane Archerfield Airport,2017-01-01 02:00:00,25.5,True,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72451,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,6443.62,QLD1,Brisbane Archerfield Airport,2021-03-17 22:00:00,19.6,False,Autumn
72452,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,6264.63,QLD1,Brisbane Archerfield Airport,2021-03-17 22:30:00,19.5,False,Autumn
72453,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,6144.16,QLD1,Brisbane Archerfield Airport,2021-03-17 23:00:00,19.4,False,Autumn
72454,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,5897.64,QLD1,Brisbane Archerfield Airport,2021-03-17 23:30:00,19.6,False,Autumn


In [35]:
df_qld_merged['Cooling'] = df_qld_merged['TEMPERATURE'].apply(lambda x: max(0, x - 24))

df_qld_merged['Heating'] = df_qld_merged['TEMPERATURE'].apply(lambda x: max(0, 20 - x))

print(df_qld_merged[['Cooling', 'Heating']].head())  # Displaying the first few rows to verify


   Cooling  Heating
0      2.5      0.0
1      2.3      0.0
2      2.0      0.0
3      1.2      0.0
4      1.5      0.0


In [36]:
df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,TOTALDEMAND,REGIONID_y,LOCATION,DATETIME,TEMPERATURE,is_weekend,season,Cooling,Heating
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,6728.99,QLD1,Brisbane Archerfield Airport,2017-01-01 00:00:00,26.5,True,Summer,2.5,0.0
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,6462.14,QLD1,Brisbane Archerfield Airport,2017-01-01 00:30:00,26.3,True,Summer,2.3,0.0
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,6352.82,QLD1,Brisbane Archerfield Airport,2017-01-01 01:00:00,26.0,True,Summer,2.0,0.0
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,6261.02,QLD1,Brisbane Archerfield Airport,2017-01-01 01:30:00,25.2,True,Summer,1.2,0.0
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,6169.66,QLD1,Brisbane Archerfield Airport,2017-01-01 02:00:00,25.5,True,Summer,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72451,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,6443.62,QLD1,Brisbane Archerfield Airport,2021-03-17 22:00:00,19.6,False,Autumn,0.0,0.4
72452,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,6264.63,QLD1,Brisbane Archerfield Airport,2021-03-17 22:30:00,19.5,False,Autumn,0.0,0.5
72453,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,6144.16,QLD1,Brisbane Archerfield Airport,2021-03-17 23:00:00,19.4,False,Autumn,0.0,0.6
72454,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,5897.64,QLD1,Brisbane Archerfield Airport,2021-03-17 23:30:00,19.6,False,Autumn,0.0,0.4


In [37]:
df_qld_merged.columns

Index(['PREDISPATCHSEQNO', 'PERIODID', 'FORECASTDEMAND', 'LASTCHANGED', 'YEAR',
       'MONTH', 'DAY', 'HOUR', 'MINUTE', 'DATETIME_y', 'TOTALDEMAND',
       'REGIONID_y', 'LOCATION', 'DATETIME', 'TEMPERATURE', 'is_weekend',
       'season', 'Cooling', 'Heating'],
      dtype='object')

In [41]:
numerical_features = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'TEMPERATURE', 'Heating', 'Cooling']
categorical_features = ['season']
binary_features = ['is_weekend']  

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features),
        ('bin', 'passthrough', binary_features)
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

X = df_qld_merged.drop(['TOTALDEMAND'], axis=1)
y = df_qld_merged['TOTALDEMAND']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')



Mean Squared Error: 356232.0119134571
R^2 Score: 0.5639904505804563


In [43]:
pipeline.fit(X_train, y_train)

ohe_feature_names = preprocessor.named_transformers_['cat'].get_feature_names(input_features=categorical_features)
feature_names_transformed = list(preprocessor.transformers_[0][2]) + list(ohe_feature_names) + binary_features

coefficients = pipeline.named_steps['regressor'].coef_

feature_importance = pd.Series(coefficients, index=feature_names_transformed)

feature_importance_sorted = feature_importance.abs().sort_values(ascending=False)

print("Feature Importance Ranked:")
print(feature_importance_sorted)


Feature Importance Ranked:
season_Winter    1.225375e+14
season_Spring    1.225375e+14
season_Autumn    1.225375e+14
season_Summer    1.225375e+14
HOUR             5.097255e+02
is_weekend       4.047461e+02
MONTH            9.699624e+01
TEMPERATURE      7.309941e+01
Cooling          7.307457e+01
Heating          7.060565e+01
YEAR             5.394109e+01
MINUTE           1.818570e+00
DAY              2.451987e-01
dtype: float64


In [46]:
entire_data_predictions = pipeline.predict(X)

df_qld_merged['PREDICTED_TOTALDEMAND'] = entire_data_predictionsb

In [47]:
df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,TOTALDEMAND,REGIONID_y,LOCATION,DATETIME,TEMPERATURE,is_weekend,season,Cooling,Heating,PREDICTED_TOTALDEMAND
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,6728.99,QLD1,Brisbane Archerfield Airport,2017-01-01 00:00:00,26.5,True,Summer,2.5,0.0,5858.296875
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,6462.14,QLD1,Brisbane Archerfield Airport,2017-01-01 00:30:00,26.3,True,Summer,2.3,0.0,5852.062500
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,6352.82,QLD1,Brisbane Archerfield Airport,2017-01-01 01:00:00,26.0,True,Summer,2.0,0.0,5907.250000
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,6261.02,QLD1,Brisbane Archerfield Airport,2017-01-01 01:30:00,25.2,True,Summer,1.2,0.0,5871.343750
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,6169.66,QLD1,Brisbane Archerfield Airport,2017-01-01 02:00:00,25.5,True,Summer,1.5,0.0,5956.187500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72451,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,6443.62,QLD1,Brisbane Archerfield Airport,2021-03-17 22:00:00,19.6,False,Autumn,0.0,0.4,6890.859375
72452,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,6264.63,QLD1,Brisbane Archerfield Airport,2021-03-17 22:30:00,19.5,False,Autumn,0.0,0.5,6895.390625
72453,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,6144.16,QLD1,Brisbane Archerfield Airport,2021-03-17 23:00:00,19.4,False,Autumn,0.0,0.6,6966.281250
72454,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,5897.64,QLD1,Brisbane Archerfield Airport,2021-03-17 23:30:00,19.6,False,Autumn,0.0,0.4,6968.140625


In [48]:
df_qld_merged['Error'] = df_qld_merged['PREDICTED_TOTALDEMAND'] - df_qld_merged['TOTALDEMAND']

df_qld_merged['Error Ratio'] = df_qld_merged['Error'] / df_qld_merged['PREDICTED_TOTALDEMAND']

df_qld_merged


,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,...,LOCATION,DATETIME,TEMPERATURE,is_weekend,season,Cooling,Heating,PREDICTED_TOTALDEMAND,Error,Error Ratio
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,...,Brisbane Archerfield Airport,2017-01-01 00:00:00,26.5,True,Summer,2.5,0.0,5858.296875,-870.693125,-0.148626
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,...,Brisbane Archerfield Airport,2017-01-01 00:30:00,26.3,True,Summer,2.3,0.0,5852.062500,-610.077500,-0.104250
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,...,Brisbane Archerfield Airport,2017-01-01 01:00:00,26.0,True,Summer,2.0,0.0,5907.250000,-445.570000,-0.075428
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,...,Brisbane Archerfield Airport,2017-01-01 01:30:00,25.2,True,Summer,1.2,0.0,5871.343750,-389.676250,-0.066369
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,...,Brisbane Archerfield Airport,2017-01-01 02:00:00,25.5,True,Summer,1.5,0.0,5956.187500,-213.472500,-0.035840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72451,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,...,Brisbane Archerfield Airport,2021-03-17 22:00:00,19.6,False,Autumn,0.0,0.4,6890.859375,447.239375,0.064903
72452,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,...,Brisbane Archerfield Airport,2021-03-17 22:30:00,19.5,False,Autumn,0.0,0.5,6895.390625,630.760625,0.091476
72453,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,...,Brisbane Archerfield Airport,2021-03-17 23:00:00,19.4,False,Autumn,0.0,0.6,6966.281250,822.121250,0.118014
72454,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,...,Brisbane Archerfield Airport,2021-03-17 23:30:00,19.6,False,Autumn,0.0,0.4,6968.140625,1070.500625,0.153628


In [50]:
average_error = df_qld_merged['Error'].mean()
average_error_ratio = df_qld_merged['Error Ratio'].mean()
std_error = df_qld_merged['Error'].std()
std_error_ratio = df_qld_merged['Error Ratio'].std()

print(f'Average Error: {average_error}')
print(f'Average Error Ratio: {average_error_ratio}')
print(f'Standard Deviation of Error: {std_error}')
print(f'Standard Deviation of Error Ratio: {std_error_ratio}')



Average Error: -1.1739412712544883
Average Error Ratio: -0.0003231972569622467
Standard Deviation of Error: 600.6124969273438
Standard Deviation of Error Ratio: 0.09481403942692497


In [54]:
df_qld_merged[['Error Ratio','Error']].describe()

,Error Ratio,Error
count,72456.000000,72456.000000
mean,-0.000323,-1.173941
std,0.094814,600.612497
min,-0.377085,-2484.786875
25%,-0.063205,-388.692656
50%,0.004607,27.651875
75%,0.065293,398.807656
max,0.332978,2032.633750


In [55]:
df_qld_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72456 entries, 0 to 72455
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PREDISPATCHSEQNO       72456 non-null  int64         
 1   PERIODID               72456 non-null  int64         
 2   FORECASTDEMAND         72456 non-null  float64       
 3   LASTCHANGED            72456 non-null  object        
 4   YEAR                   72456 non-null  int64         
 5   MONTH                  72456 non-null  int64         
 6   DAY                    72456 non-null  int64         
 7   HOUR                   72456 non-null  int64         
 8   MINUTE                 72456 non-null  int64         
 9   DATETIME_y             72456 non-null  datetime64[ns]
 10  TOTALDEMAND            72456 non-null  float64       
 11  REGIONID_y             72456 non-null  object        
 12  LOCATION               72456 non-null  object        
 13  D

## Creating a NN based onthe intra day data

In [76]:
from sklearn.neural_network import MLPRegressor

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000, random_state=42))
])

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


Mean Squared Error: 66750.91051280018
R^2 Score: 0.9183003395464054


/Users/joshuaevans/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [77]:
df_qld_merged['PREDICTED_TOTALDEMAND_NN'] = pipeline.predict(X)

df_qld_merged['Error_NN'] = df_qld_merged['PREDICTED_TOTALDEMAND_NN'] - df_qld_merged['TOTALDEMAND']

df_qld_merged['Error Ratio_NN'] = df_qld_merged['Error_NN'] / df_qld_merged['PREDICTED_TOTALDEMAND_NN']

df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,...,Cooling,Heating,PREDICTED_TOTALDEMAND,Error,Error Ratio,PREDICTED_TOTALDEMAND_NN,Error_NN,Error Ratio_NN,Error_Prof,Error Ratio_Prof
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,...,2.5,0.0,5858.296875,-870.693125,-0.148626,6152.286419,-576.703581,-0.093738,-303.63,-0.047255
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,...,2.3,0.0,5852.062500,-610.077500,-0.104250,5984.720738,-477.419262,-0.079773,-268.80,-0.043401
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,...,2.0,0.0,5907.250000,-445.570000,-0.075428,5971.754169,-381.065831,-0.063811,-336.37,-0.055908
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,...,1.2,0.0,5871.343750,-389.676250,-0.066369,5825.826713,-435.193287,-0.074701,-401.79,-0.068574
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,...,1.5,0.0,5956.187500,-213.472500,-0.035840,5795.064330,-374.595670,-0.064640,-402.51,-0.069794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72451,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,...,0.0,0.4,6890.859375,447.239375,0.064903,6619.998574,176.378574,0.026643,219.02,0.032873
72452,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,...,0.0,0.5,6895.390625,630.760625,0.091476,6519.015531,254.385531,0.039022,253.19,0.038846
72453,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,...,0.0,0.6,6966.281250,822.121250,0.118014,6316.923061,172.763061,0.027349,222.91,0.035010
72454,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,...,0.0,0.4,6968.140625,1070.500625,0.153628,6216.758048,319.118048,0.051332,267.41,0.043375


In [78]:
df_qld_merged['Error_Prof'] = df_qld_merged['FORECASTDEMAND'] - df_qld_merged['TOTALDEMAND']

df_qld_merged['Error Ratio_Prof'] = df_qld_merged['Error_Prof'] / df_qld_merged['FORECASTDEMAND']

df_qld_merged

,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,YEAR,MONTH,DAY,HOUR,MINUTE,DATETIME_y,...,Cooling,Heating,PREDICTED_TOTALDEMAND,Error,Error Ratio,PREDICTED_TOTALDEMAND_NN,Error_NN,Error Ratio_NN,Error_Prof,Error Ratio_Prof
0,2016123018,71,6425.36,2016-12-30 12:31:25,2017,1,1,0,0,2017-01-01 00:00:00,...,2.5,0.0,5858.296875,-870.693125,-0.148626,6152.286419,-576.703581,-0.093738,-303.63,-0.047255
1,2016123018,72,6193.34,2016-12-30 12:31:25,2017,1,1,0,30,2017-01-01 00:30:00,...,2.3,0.0,5852.062500,-610.077500,-0.104250,5984.720738,-477.419262,-0.079773,-268.80,-0.043401
2,2016123018,73,6016.45,2016-12-30 12:31:25,2017,1,1,1,0,2017-01-01 01:00:00,...,2.0,0.0,5907.250000,-445.570000,-0.075428,5971.754169,-381.065831,-0.063811,-336.37,-0.055908
3,2016123018,74,5859.23,2016-12-30 12:31:25,2017,1,1,1,30,2017-01-01 01:30:00,...,1.2,0.0,5871.343750,-389.676250,-0.066369,5825.826713,-435.193287,-0.074701,-401.79,-0.068574
4,2016123018,75,5767.15,2016-12-30 12:31:25,2017,1,1,2,0,2017-01-01 02:00:00,...,1.5,0.0,5956.187500,-213.472500,-0.035840,5795.064330,-374.595670,-0.064640,-402.51,-0.069794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72451,2021031618,67,6662.64,2021-03-16 12:31:57,2021,3,17,22,0,2021-03-17 22:00:00,...,0.0,0.4,6890.859375,447.239375,0.064903,6619.998574,176.378574,0.026643,219.02,0.032873
72452,2021031618,68,6517.82,2021-03-16 12:31:57,2021,3,17,22,30,2021-03-17 22:30:00,...,0.0,0.5,6895.390625,630.760625,0.091476,6519.015531,254.385531,0.039022,253.19,0.038846
72453,2021031618,69,6367.07,2021-03-16 12:31:57,2021,3,17,23,0,2021-03-17 23:00:00,...,0.0,0.6,6966.281250,822.121250,0.118014,6316.923061,172.763061,0.027349,222.91,0.035010
72454,2021031618,70,6165.05,2021-03-16 12:31:57,2021,3,17,23,30,2021-03-17 23:30:00,...,0.0,0.4,6968.140625,1070.500625,0.153628,6216.758048,319.118048,0.051332,267.41,0.043375


In [79]:
df_qld_merged[['Error Ratio_NN','Error_NN','Error Ratio','Error', 'Error_Prof','Error Ratio_Prof']].describe()

,Error Ratio_NN,Error_NN,Error Ratio,Error,Error_Prof,Error Ratio_Prof
count,72456.000000,72456.000000,72456.000000,72456.000000,72456.000000,72456.000000
mean,0.001428,9.276031,-0.000323,-1.173941,-8.397029,-0.001468
std,0.039722,254.662109,0.094814,600.612497,158.492563,0.025199
min,-0.257477,-1619.783042,-0.377085,-2484.786875,-918.000000,-0.187521
25%,-0.019687,-119.601149,-0.063205,-388.692656,-91.660000,-0.014952
50%,0.001513,9.121971,0.004607,27.651875,-2.460000,-0.000401
75%,0.023098,141.135250,0.065293,398.807656,81.720000,0.013528
max,0.251713,1793.611290,0.332978,2032.633750,1170.470000,0.149933


In [81]:
mape = (df_qld_merged['Error_NN'].abs() / df_qld_merged['TOTALDEMAND'].abs()).mean() * 100

print(f'MAPE: {mape}%')

MAPE: 2.9116994174081383%
